In [1]:
import numpy as np
import sys
import os

ml_args_raster_low_res = './raster/low_res'
ml_args_raster_high_res = './raster/high_res'
ml_args_count = 13597

def ml_raster_import( ml_path ):

    # check consistency #
    if ( not os.path.exists( ml_path ) ):

        # send message #
        sys.exit( 'turing : error : unable to access raster' )

    # retrieve raster size #
    ml_size = os.path.getsize( ml_path )

    # compute raster width #
    ml_width = int( round( ml_size ** ( 1.0 / 3.0 ) ) )

    # import raster data #
    with open( ml_path, 'rb' ) as ml_file:

        # read raster bytes #
        ml_byte = ml_file.read( ml_size )

    # convert to numpy array #
    ml_data = np.frombuffer( ml_byte, dtype=np.uint8 )

    # return raster array #
    return ml_data.reshape( ml_width, ml_width, ml_width )

##
##  script - raster convertion
##

def ml_raster_convert( ml_raster ):

    # check consistency #
    if ( ( ml_raster.shape[0] != ml_raster.shape[1] ) or ( ml_raster.shape[0] != ml_raster.shape[2] ) ):

        # send message #
        sys.exit( 'turing : error : raster not consistent' )

    # retrieve raster width #
    ml_width = ml_raster.shape[0]

    # initialise array #
    ml_data = np.empty( ( 0, 3 ), dtype=np.uint8 )

    # parsing raster array #
    for ml_x in range( ml_width ):

        # parsing raster array #
        for ml_y in range( ml_width ):

            # parsing raster array #
            for ml_z in range( ml_width ):

                # check raster element #
                if ( ml_raster[ml_x, ml_y, ml_z] != 0 ):

                    # compute element coordinates #
                    ml_r = ml_x / ml_width
                    ml_s = ml_y / ml_width
                    ml_t = ml_z / ml_width

                    # append element #
                    ml_data = np.append( ml_data, np.array( [[ ml_r, ml_s, ml_t ]] ), axis=0 )

    # return converted array #
    return( ml_data )

In [3]:
def get_ml_data(ml_args_raster):

    # It will take circa 20 min to get the 13k values so we just take 100
    ml_data = []
    
    for raster_id in range(100):
        
        raster_path = ml_args_raster + '/raster-{:06d}.ras'.format(raster_id)
    
        # import raster array #
        data = ml_raster_import(raster_path)

        # convert raster to point set #
        #data = ml_raster_convert(data)
    
        ml_data.append(data)
        
    return ml_data

ml_data_low_res = get_ml_data(ml_args_raster_low_res)
ml_data_high_res = get_ml_data(ml_args_raster_high_res)

In [4]:
# points are defined in 3D coordinates
ml_data_low_res[0]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 0, 1],
        ...,
        [1, 1, 1, ..., 

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [215]:
class CloudPointDataset(Dataset):

    def __init__(self, thin_dataset, thick_dataset, train, transform=None):
        """
        Args:
            mat_file (string): Path to the mat file with the data
            train (boolean): Is it the training data or the test data
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.X = thin_dataset
        self.Y = thick_dataset
        self.X = np.asarray(self.X).astype(float)
        print(self.X)
        self.Y = np.asarray(self.Y).astype(float).squeeze().squeeze()
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(self.X, self.Y, test_size=0.3, random_state=1)
        # self.Xtrain, self.Xtest  = np.asarray(Xtrain).astype(float), np.asarray(Xtest).astype(float)
        # self.Ytrain, self.Ytest = np.asarray(Ytrain).astype(float), np.asarray(Ytest).astype(float)
        self.Xtrain, self.Xtest = Xtrain, Xtest
        self.Ytrain, self.Ytest = Ytrain, Ytest
        # self.Ytrain, self.Ytest = self.Ytrain.squeeze()-1, self.Ytest.squeeze()-1
        self.train = train
        self.transform = transform

    def __len__(self):
        if self.train:
            return (self.Xtrain.shape[0])
        else:
            return (self.Xtest.shape[0])

    def __getitem__(self, idx):
        
        if self.train:
            #img = self.Xtrain[idx,:].transpose((2, 1, 0))
            #sample = {'image': img, 'label': self.Ytrain[idx]}
            sample = {'image': self.Xtrain[idx,:], 'label': self.Ytrain[idx,:]}
        else:
            #img = self.Xtest[idx,:].transpose((2, 1, 0))
            #sample = {'image': img, 'label': self.Ytest[idx]}
            sample = {'image': self.Xtest[idx,:], 'label': self.Ytest[idx,:]}
            
        if self.transform:
            sample = self.transform(sample)

        return sample

In [216]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        raster, label = sample['image'], sample['label']
        raster = torch.from_numpy(raster).float()
        raster = raster.expand(1, 16, 16, 16)
        return {'image': raster,
                'label': label}

In [217]:
trainset = CloudPointDataset(ml_data_low_res, ml_data_high_res, train=True, transform = ToTensor())
#print((trainset[0]['image']).expand(3, 16, 16, 16).size())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=3, shuffle=True, num_workers=2)

testset = CloudPointDataset(ml_data_low_res, ml_data_high_res, train=False, transform=ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=3, shuffle=False, num_workers=2)

[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[1. 1. 1. ... 1. 1. 1.]
   [1. 1. 1. ... 1. 1. 1.]
   [1. 1. 1. ... 1. 0. 1.]
   ...
   [1. 1. 1. ... 0. 0. 1.]
   [1. 1. 1. ... 0. 0. 1.]
   [1. 1. 1. ... 0. 0. 1.]]

  [[0. 0. 1. ... 0. 0. 0.]
   [0. 0. 1. ... 1. 1. 1.]
   [0. 1. 1. ... 1. 1. 1.]
   ...
   [1. 1. 0. ... 1. 1. 1.]
   [0. 0. 0. ... 1. 1. 

In [218]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(1, 3, 5)
        self.padding = nn.ReplicationPad3d(6)
        #self.unpool = nn.MaxUnpool3d(4)
        self.conv2 = nn.Conv3d(3, 1, 5)

    def forward(self, x):
        x = self.padding(x)
        x = F.relu(self.conv1(x))
        x = self.padding(x)
        x = F.relu(self.conv2(x))

        return x

net = Net()

In [219]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [220]:
from torch.autograd import Variable
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        images, labels = data['image'], data['label'].long()

        # wrap them in Variable
        images, labels = Variable(images), Variable(labels)

        # clear the gradients of the variables
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        print(outputs.size())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')

torch.Size([3, 1, 32, 32, 32])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/THNN/generic/SpatialClassNLLCriterion.c:110